# VASP WAVECAR分析  
首先从[GitHub](https://github.com/QijingZheng/VaspBandUnfolding)Fork了已有的库，目前仅需要其中的`vaspwfc`模块中的`vaspwfc`类。在一开始import所需要的部分即可。  
`from vaspwfc impoirt vaspwfc`  

In [1]:
import numpy as np
import matplotlib.pyplot as plt 

from vaspwfc import vaspwfc

利用Bulk Graphene的WAVECAR生成类。路径Example_Klaus/WAVECAR。  
`vaspwfc(fnm='WAVECAR', lsorbit=False, lgamma=False,gamma_half='x', omp_num_threads=1)`  
- `fnm`:filename of WAVECAR, you could use the relative path representation, e.g. './Example/WAVECAR'  
- 针对Gamma Only的WAVECAR，需要将`lgamma`设置为`True`
```python
# For VASP <= 5.2.x, check
# which FFT VASP uses by the following command:
#
#     $ grep 'use.* FFT for wave' OUTCAR
#
# Then
#
#     # for parallel FFT, VASP <= 5.2.x
#     pswfc = vaspwfc('WAVECAR', lgamma=True, gamma_half='z')
#
#     # for serial FFT, VASP <= 5.2.x
#     pswfc = vaspwfc('WAVECAR', lgamma=True, gamma_half='x')
#
# For VASP >= 5.4, WAVECAR is written with x-direction half grid regardless of
# parallel or serial FFT.
#
#     # "gamma_half" default to "x" for VASP >= 5.4
#     pswfc = vaspwfc('WAVECAR', lgamma=True, gamma_half='x')
```
本次计算中使用的是VASP 5.4.4，因此不需要考虑这个参数的问题。 
- `lsorbit`针对non-collinear计算需要设置为`True`，一般情况下设置为`False`，SOC开启的情况下是否需要设置为True？

In [10]:
wav = vaspwfc("./Example_Klaus/WAVECAR")
print("No. K-POINTS: {:4d} \nNo. Bands: {:4d} \nENCUT: {:6.2f}".format(wav._nkpts,wav._nbands,wav._encut))


No. K-POINTS:  120 
No. Bands:   48 
ENCUT: 600.00


`gvectors(ikpt=1, force_Gamma=False, check_consistency=True)`方法可以根据输入的`ikpt`(k点的索引)，生成所有满足(G + k)**2 / 2 < ENCUT的G-vectors。所有的k点坐标可以通过调用`_kvecs`属性查看。

In [23]:
print("ikpt \t K-vectors")
print("-"*30) 
for index in range(wav._nkpts):
    print("\t".join([str(index+1),str(wav._kvecs[index])]))

ikpt 	 K-vectors
------------------------------
1	[0. 0. 0.]
2	[0.01282051 0.         0.        ]
3	[0.02564103 0.         0.        ]
4	[0.03846154 0.         0.        ]
5	[0.05128205 0.         0.        ]
6	[0.06410256 0.         0.        ]
7	[0.07692308 0.         0.        ]
8	[0.08974359 0.         0.        ]
9	[0.1025641 0.        0.       ]
10	[0.11538462 0.         0.        ]
11	[0.12820513 0.         0.        ]
12	[0.14102564 0.         0.        ]
13	[0.15384615 0.         0.        ]
14	[0.16666667 0.         0.        ]
15	[0.17948718 0.         0.        ]
16	[0.19230769 0.         0.        ]
17	[0.20512821 0.         0.        ]
18	[0.21794872 0.         0.        ]
19	[0.23076923 0.         0.        ]
20	[0.24358974 0.         0.        ]
21	[0.25641026 0.         0.        ]
22	[0.26923077 0.         0.        ]
23	[0.28205128 0.         0.        ]
24	[0.29487179 0.         0.        ]
25	[0.30769231 0.         0.        ]
26	[0.32051282 0.         0.        ]
